<a href="https://colab.research.google.com/github/paroonk/optimization-mip-project/blob/main/python_mip_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import


In [ ]:
!pip install mip
import numpy as np
from mip import *

     |████████████████████████████████| 48.0MB 74kB/s 


# Basic Linear Programming

In [ ]:
activity = ['pushup', 'running']
time_used = [0.2, 10]
calories_burned = [3, 130]
limit_time = 10

n, I = len(activity), range(len(activity))

model = Model()

# Decision variables
x = [model.add_var(var_type=CONTINUOUS, name=activity[i]) for i in I]

# Objective
model.objective = maximize(xsum(calories_burned[i] * x[i] for i in I ))

# Constraints
model += xsum(time_used[i] * x[i] for i in I) <= limit_time

# Optimize and Result
status = model.optimize()
if status == OptimizationStatus.OPTIMAL:
  print('Optimal solution cost {} found'.format(model.objective_value))
elif status == OptimizationStatus.FEASIBLE:
  print('Sol.cost {} found, best possible: {}'.format(model.objective_value, model.objective_bound))
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
  print('No feasible solution found, lower bound is: {}'.format(model.objective_bound))
else:
  print('Infeasible')
    
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
  for v in model.vars:
    # if abs(v.x) > 1e-6: # only printing non-zeros
      print('{} : {}'.format(v.name, v.x))

# Check
print('\nCheck results')
print('Total time used: {} <= {}'.format(sum(time_used[i] * x[i].x for i in I), limit_time))

Optimal solution cost 150.0 found
pushup : 50.0
running : 0.0

Check results
Total time used: 10.0 <= 10


# The 0/1 Knapsack Problem

As a first example, consider the solution of the 0/1 knapsack problem: given a set I of items, each one with a weight wi and estimated profit pi, one wants to select a subset with maximum profit such that the summation of the weights of the selected items is less or equal to the knapsack capacity c. Considering a set of decision binary variables xi that receive value 1 if the i-th item is selected, or 0 if not.

In [ ]:
p = [10, 13, 18, 31, 7, 15]
w = [11, 15, 20, 35, 10, 33]
limit_w = 47
I = range(len(w))

model = Model()

# decision variables
x = [model.add_var(var_type=BINARY) for i in I]

# objective function
model.objective = maximize(xsum(p[i] * x[i] for i in I))

# constraint
model += xsum(w[i] * x[i] for i in I) <= limit_w
    
# optimizing
status = model.optimize()

if status == OptimizationStatus.OPTIMAL:
  print('Optimal solution cost {} found'.format(model.objective_value))
elif status == OptimizationStatus.FEASIBLE:
  print('Sol.cost {} found, best possible: {}'.format(model.objective_value, model.objective_bound))
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
  print('No feasible solution found, lower bound is: {}'.format(model.objective_bound))
else:
  print('Infeasible')
    
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
  for v in model.vars:
    # if abs(v.x) > 1e-6: # only printing non-zeros
      print('{} : {}'.format(v.name, v.x))

# Check
print('\nCheck results')
print('Total weight used: {} <= {}'.format(sum(w[i] * x[i].x for i in I), limit_w))

Optimal solution cost 41.0 found
var(0) : 1.0
var(1) : 0.0
var(2) : 0.0
var(3) : 1.0
var(4) : 0.0
var(5) : 0.0

Check results
Total weight used: 46.0 <= 47


# The Traveling Salesman Problem
The traveling salesman problem (TSP) is one of the most studied combinatorial optimization problems, with the first computational studies dating back to the 50s. To to illustrate this problem, consider that you will spend some time in Belgium and wish to visit some of its main tourist attractions, depicted in the map bellow:

![](https://docs.python-mip.com/en/latest/_images/belgium-tourism-14.png)

You want to find the shortest possible tour to visit all these places. More formally, considering n points V={0,…,n−1} and a distance matrix Dn×n with elements ci,j∈R+, a solution consists in a set of exactly n (origin, destination) pairs indicating the itinerary of your trip.

In [ ]:
from itertools import product
from sys import stdout as out

# names of places to visit
places = ['Antwerp', 'Bruges', 'C-Mine', 'Dinant', 'Ghent',
          'Grand-Place de Bruxelles', 'Hasselt', 'Leuven',
          'Mechelen', 'Mons', 'Montagne de Bueren', 'Namur',
          'Remouchamps', 'Waterloo']

# distances in an upper triangular matrix
dists = [[83, 81, 113, 52, 42, 73, 44, 23, 91, 105, 90, 124, 57],
         [161, 160, 39, 89, 151, 110, 90, 99, 177, 143, 193, 100],
         [90, 125, 82, 13, 57, 71, 123, 38, 72, 59, 82],
         [123, 77, 81, 71, 91, 72, 64, 24, 62, 63],
         [51, 114, 72, 54, 69, 139, 105, 155, 62],
         [70, 25, 22, 52, 90, 56, 105, 16],
         [45, 61, 111, 36, 61, 57, 70],
         [23, 71, 67, 48, 85, 29],
         [74, 89, 69, 107, 36],
         [117, 65, 125, 43],
         [54, 22, 84],
         [60, 44],
         [97]]

# number of nodes and list of vertices
n, V = len(places), set(range(len(places)))

# distances matrix
c = [[0 if i == j
      else dists[i][j-i-1] if j > i
      else dists[j][i-j-1]
      for j in V] for i in V]

model = Model()

# decision variables
# binary variables indicating if arc (i,j) is used on the route or not
x = [[model.add_var(var_type=BINARY) for j in V] for i in V]

# continuous variable to prevent subtours: each city will have a
# different sequential id in the planned route except the first one
y = [model.add_var() for i in V]

# objective function
model.objective = minimize(xsum(c[i][j] * x[i][j] for j in V for i in V))

# constraint
# leave each city only once
for i in V:
  model += xsum(x[i][j] for j in V - {i}) == 1

# enter each city only once
for i in V:
  model += xsum(x[j][i] for j in V - {i}) == 1

# subtour elimination
for (i, j) in product(V - {0}, V - {0}):
  if i != j:
    model += y[i] - (n + 1) * x[i][j] >= y[j] - n
    
# optimizing
status = model.optimize()
if status == OptimizationStatus.OPTIMAL:
  print('Optimal solution cost {} found'.format(model.objective_value))
elif status == OptimizationStatus.FEASIBLE:
  print('Sol.cost {} found, best possible: {}'.format(model.objective_value, model.objective_bound))
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
  print('No feasible solution found, lower bound is: {}'.format(model.objective_bound))
else:
  print('Infeasible')

# checking if a solution was found
if model.num_solutions:
  out.write('route with total distance %g found:\n%s' % (model.objective_value, places[0]))
  nc = 0
  start_c = nc
  remain_c = V - {nc}
  while True:
    nc = [i for i in V if x[nc][i].x >= 0.99][0]
    out.write(' -> %s' % places[nc])
    remain_c = remain_c - {nc}
    if nc == start_c:
      if len(remain_c) == 0:
        break
      else:
        nc = list(remain_c)[0]
        start_c = nc
        out.write('\n%s' % places[nc])
  out.write('\n')

Optimal solution cost 547.0 found
route with total distance 547 found:
Antwerp -> Bruges -> Ghent -> Grand-Place de Bruxelles -> Waterloo -> Mons -> Namur -> Dinant -> Remouchamps -> Montagne de Bueren -> C-Mine -> Hasselt -> Leuven -> Mechelen -> Antwerp


# n-Queens
In the n-queens puzzle n chess queens should to be placed in a board with n×n cells in a way that no queen can attack another, i.e., there must be at most one queen per row, column and diagonal. This is a constraint satisfaction problem: any feasible solution is acceptable and no objective function is defined.

In [ ]:
from sys import stdout as out

# number of queens
n = 40
N = range(n)

model = Model()

# decision variables
x = [[model.add_var(var_type=BINARY, name='x({},{})'.format(i, j)) for j in N] for i in N]

# objective function
model.objective = maximize(xsum(x[i][j] for j in N for i in N))

# constraint
# one per row
for i in N:
  model += xsum(x[i][j] for j in N) <= 1, 'row({})'.format(i)

# one per column
for j in N:
  model += xsum(x[i][j] for i in N) <= 1, 'col({})'.format(j)

# diagonal \
for p, k in enumerate(range(2 - n, n - 2 + 1)):
  model += xsum(x[i][i - k] for i in N if 0 <= i - k < n) <= 1, 'diag1({})'.format(p)

# diagonal /
for p, k in enumerate(range(2 - n, n - 2 + 1)):
  model += xsum(x[i][-i + k + 3] for i in N if 0 <= -i + k + 3 + 1 < n) <= 1, 'diag2({})'.format(p)
    
# optimizing
status = model.optimize()
if status == OptimizationStatus.OPTIMAL:
  print('Optimal solution cost {} found'.format(model.objective_value))
elif status == OptimizationStatus.FEASIBLE:
  print('Sol.cost {} found, best possible: {}'.format(model.objective_value, model.objective_bound))
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
  print('No feasible solution found, lower bound is: {}'.format(model.objective_bound))
else:
  print('Infeasible')
        
# checking if a solution was found
if model.num_solutions:
  out.write('\n')
  for i, v in enumerate(model.vars):
    out.write('O ' if v.x >= 0.99 else '. ')
    if i % n == n-1:
      out.write('\n')

Optimal solution cost 40.0 found

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . O . . . . . . . . . 
. . . . . . . . . . . . . O . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . O . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . O . . . . . . . . 
. . . . . . . . O . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . O . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . O . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . O 
. . . . . . . . . . . . . . . . O . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . O . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . O . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . O . . . . . . . . . . .

# Frequency Assignment
The design of wireless networks, such as cell phone networks, involves assigning communication frequencies to devices. These communication frequencies can be separated into channels. The geographical area covered by a network can be divided into hexagonal cells, where each cell has a base station that covers a given area. Each cell requires a different number of channels, based on usage statistics and each cell has a set of neighbor cells, based on the geographical distances. The design of an efficient mobile network involves selecting subsets of channels for each cell, avoiding interference between calls in the same cell and in neighboring cells. Also, for economical reasons, the total bandwidth in use must be minimized, i.e., the total number of different channels used. One of the first real cases discussed in literature are the Philadelphia instances, with the structure depicted bellow:

![](https://docs.python-mip.com/en/latest/_images/bmcpsmall.png)

Each cell has a demand with the required number of channels drawn at the center of the hexagon, and a sequential id at the top left corner. Also, in this example, each cell has a set of at most 6 adjacent neighboring cells (distance 1). The largest demand (8) occurs on cell 2. This cell has the following adjacent cells, with distance 1: (1, 6). The minimum distances between channels in the same cell in this example is 3 and channels in neighbor cells should differ by at least 2 units.

A generalization of this problem (not restricted to the hexagonal topology), is the Bandwidth Multicoloring Problem (BMCP), which has the following input data:

N:
set of cells, numbered from 1 to n;

ri∈Z+:
demand of cell i∈N, i.e., the required number of channels;

di,j∈Z+:
minimum distance between channels assigned to nodes i and j, di,i indicates the minimum distance between different channels allocated to the same cell.

In [ ]:
from itertools import product

# number of channels per node
r = [3, 5, 8, 3, 6, 5, 7, 3]
N = range(len(r))

# distance between channels in the same node (i, i) and in adjacent nodes
#     0  1  2  3  4  5  6  7
d = [[3, 2, 0, 0, 2, 2, 0, 0],   # 0
     [2, 3, 2, 0, 0, 2, 2, 0],   # 1
     [0, 2, 3, 0, 0, 0, 3, 0],   # 2
     [0, 0, 0, 3, 2, 0, 0, 2],   # 3
     [2, 0, 0, 2, 3, 2, 0, 0],   # 4
     [2, 2, 0, 0, 2, 3, 2, 0],   # 5
     [0, 2, 2, 0, 0, 2, 3, 0],   # 6
     [0, 0, 0, 2, 0, 0, 0, 3]]   # 7

# in complete applications this upper bound should be obtained from a feasible solution produced with some heuristic
U = range(sum(d[i][j] for (i, j) in product(N, N)) + sum(el for el in r))

model = Model()

# decision variables
x = [[model.add_var('x({},{})'.format(i, c), var_type=BINARY) for c in U] for i in N]

z = model.add_var('z')

# objective function
model.objective = minimize(z)

# constraint
for i in N:
  model += xsum(x[i][c] for c in U) == r[i]

for i, j, c1, c2 in product(N, N, U, U):
  if i != j and c1 <= c2 < c1 + d[i][j]:
    model += x[i][c1] + x[j][c2] <= 1

for i, c1, c2 in product(N, U, U):
  if c1 < c2 < c1 + d[i][i]:
    model += x[i][c1] + x[i][c2] <= 1

for i, c in product(N, U):
  model += z >= (c+1) * x[i][c]
    
# optimizing
status = model.optimize(max_nodes=30)

if status == OptimizationStatus.OPTIMAL:
  print('Optimal solution cost {} found'.format(model.objective_value))
elif status == OptimizationStatus.FEASIBLE:
  print('Sol.cost {} found, best possible: {}'.format(model.objective_value, model.objective_bound))
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
  print('No feasible solution found, lower bound is: {}'.format(model.objective_bound))
else:
  print('Infeasible')
    
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
  for i in N:
    print('Channels of node %d: %s' % (i, [c for c in U if x[i][c].x >= 0.99]))


Optimal solution cost 41.0 found
Channels of node 0: [0, 6, 12]
Channels of node 1: [4, 10, 16, 22, 38]
Channels of node 2: [2, 8, 14, 20, 26, 31, 36, 40]
Channels of node 3: [1, 6, 13]
Channels of node 4: [4, 15, 18, 22, 29, 34]
Channels of node 5: [2, 8, 20, 26, 31]
Channels of node 6: [0, 6, 12, 18, 24, 29, 34]
Channels of node 7: [8, 11, 18]


# Resource Constrained Project Scheduling
The Resource-Constrained Project Scheduling Problem (RCPSP) is a combinatorial optimization problem that consists of finding a feasible scheduling for a set of n jobs subject to resource and precedence constraints. Each job has a processing time, a set of successors jobs and a required amount of different resources. Resources may be scarce but are renewable at each time period. Precedence constraints between jobs mean that no jobs may start before all its predecessors are completed. The jobs must be scheduled non-preemptively, i.e., once started, their processing cannot be interrupted.

In addition to the jobs that belong to the project, the set J contains jobs 0 and n+1, which are dummy jobs that represent the beginning and the end of the planning, respectively. The processing time for the dummy jobs is always zero and these jobs do not consume resources.

A binary programming formulation was proposed by Pritsker et al. [PWW69]. In this formulation, decision variables xjt=1 if job j is assigned to begin at time t; otherwise, xjt=0. All jobs must finish in a single instant of time without violating precedence constraints while respecting the amount of available resources.

An instance is shown below. The figure shows a graph where jobs in J are represented by nodes and precedence relations S are represented by directed edges. The time-consumption pj and all information concerning resource consumption u(j,r) are included next to the graph. This instance contains 10 jobs and 2 renewable resources, R={r1,r2}, where c1 = 6 and c2 = 8. Finally, a valid (but weak) upper bound on the time horizon T can be estimated by summing the duration of all jobs.

![](https://docs.python-mip.com/en/latest/_images/rcpsp.png)



In [ ]:
from itertools import product

use_time = [0, 3, 2, 5, 4, 2, 3, 4, 2, 4, 6, 0]

req_resource = [[0, 0], [5, 1], [0, 4], [1, 4], [1, 3], [3, 2], [3, 1], [2, 4],
                [4, 0], [5, 2], [2, 5], [0, 0]]
avail_resource = [6, 8]

PREDECESSOR = [[0, 1], [0, 2], [0, 3], [1, 4], [1, 5], [2, 9], [2, 10], [3, 8], [4, 6],
               [4, 7], [5, 9], [5, 10], [6, 8], [6, 9], [7, 8], [8, 11], [9, 11], [10, 11]]

(RESOURCE, JOB, TIME) = (range(len(avail_resource)), range(len(use_time)), range(sum(use_time)))

model = Model()

# decision variables
x = [[model.add_var(name='x({},{})'.format(j, t), var_type=BINARY) for t in TIME] for j in JOB]

# objective function
model.objective = minimize(xsum(t * x[-1][t] for t in TIME))

# constraint
for j in JOB:
  model += xsum(x[j][t] for t in TIME) == 1

for (r, t) in product(RESOURCE, TIME):
  model += xsum(req_resource[j][r] * x[j][t2] for j in JOB for t2 in range(max(0, t - use_time[j] + 1), t + 1)) <= avail_resource[r]

for (pre, j) in PREDECESSOR:
  model += xsum(t * x[j][t] - t * x[pre][t] for t in TIME) >= use_time[j]
    
# optimizing
status = model.optimize(max_nodes=30)

if status == OptimizationStatus.OPTIMAL:
  print('Optimal solution cost {} found'.format(model.objective_value))
elif status == OptimizationStatus.FEASIBLE:
  print('Sol.cost {} found, best possible: {}'.format(model.objective_value, model.objective_bound))
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
  print('No feasible solution found, lower bound is: {}'.format(model.objective_bound))
else:
  print('Infeasible')
    
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
  print('\nSchedule: ')
  for (j, t) in product(JOB, TIME):
    if x[j][t].x >= 0.99:
      print('Job {}: begins at t={} and finishes at t={}'.format(j, t, t + use_time[j]))
  print('Makespan = {}'.format(model.objective_value))

Sol.cost 21.0 found, best possible: 16.50068704218902

Schedule: 
Job 0: begins at t=0 and finishes at t=0
Job 1: begins at t=0 and finishes at t=3
Job 2: begins at t=2 and finishes at t=4
Job 3: begins at t=13 and finishes at t=18
Job 4: begins at t=5 and finishes at t=9
Job 5: begins at t=3 and finishes at t=5
Job 6: begins at t=10 and finishes at t=13
Job 7: begins at t=11 and finishes at t=15
Job 8: begins at t=19 and finishes at t=21
Job 9: begins at t=15 and finishes at t=19
Job 10: begins at t=5 and finishes at t=11
Job 11: begins at t=21 and finishes at t=21
Makespan = 21.0
